<div style="color: #ffffff; background-color: #2979ff; border-bottom: 1px solid #004ecb; padding: 2em;">
<b style="font-size: 20pt;">TP4: Stochastic gradient & co</b> <br> <br>
<b style="font-size: 15pt; color: #bbdefb">Ensimag 2A — Special grading lab</b>
</div>

**Student(s): Name1 (and Name2)**

In [ ]:
# Automatic reload of local libraries
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%matplotlib inline

>**[I. Introduction](#I.-Introduction)**<br> 
**[II. The optimization problem](#II.-The-optimization-problem)**<br>
**[III. Implementing the solvers](#III.-Implementing-the-solvers)**<br>
&nbsp; &nbsp; &nbsp;[GD (Gradient descent)](#GD-(Gradient-descent))<br>
&nbsp; &nbsp; &nbsp;[SGD (Stochastic Gradient Descent)](#SGD-(Stochastic-Gradient-Descent)<br>
&nbsp; &nbsp; &nbsp;[SAGA (Stochastic Average Gradient "Amélioré")](#SAGA-(Stochastic-Average-Gradient-"Amélioré")<br>
&nbsp; &nbsp; &nbsp;[Comparison](#Comparison)<br>
**[IV. Regularization](#IV.-Regularization)**<br>
**[V. Further development](#V.-Further-development)**<br>

In this lab, you will implement optimization algorithms that we've studied together and use them to fit a logistic regression classification model. Parts of this lab intersect TP2 and 3.

You will be implementing a simplified version of [`sklearn.linear_model.LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). 

Part of the work has already been done for you (see the `\lib` folder), so you'll only have to implement the `##### TODO (i) #####` sections of the code. Whenever you complete such a section, we ask you to **copy it in this notebook**. We want you to **only send us a pdf or html export of this notebook** (so we won't be able to see your work if it's not in this notebook). 

Report in groups of 1 to 2 students. <br>

<br>
<div style="color: #ffffff; background-color: #2979ff; border-bottom: 5px solid #004ecb; padding: 2em;"> 
<h1>I. Introduction</h1>
</div>

As in previous labs, we consider the [Student Performance](http://archive.ics.uci.edu/ml/datasets/Student+Performance) dataset. This dataset contains student demographic, social, school-related features, how many classes they've missed, etc. 

The goal we give ourselves is to predict whether a student will pass (final grade > 12).

We denote $A \in \mathcal{M}_{n,d}(\mathbb{R})$ the feature matrix, containing $n$ samples with $d$ features, and $b \in \{-1,1\}^n$ the label vector where $-1$ encodes `fail` and $1$ encodes `pass`.

The data has already been preprocessed: You can check ```\lib\datasets.py``` if you're curious. The following loads the dataset:

In [ ]:
from lib.datasets import load_student_data

A_train, A_test, b_train, b_test = load_student_data('data/student-mat.csv', split=0.25)

As mentioned previously, you're going to implement a simplified version of [`sklearn.linear_model.LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). So let's first see the type of output it is supposed to give. 

<div style="background-color: #e1f5fe; border-left: 5px solid #2962ff; padding: 0.5em;"> 
<b style="color: #2962ff;">Question:</b>  
    <ul>
        <li> Import <a href="https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html">sklearn.linear_model.LogisticRegression </a>. </li>
        <li> Create an instance of sklearn's logistic regression classifiers with SAGA as its solver. (You may decrease the target accuracy to 0.001) </li>
        <li> Fit the classifier on the training data and compute the score of the fitted model on the test set.</li>
    </ul>
</div>

<br>
<div style="color: #ffffff; background-color: #2979ff; border-bottom: 5px solid #004ecb; padding: 2em;"> 
<h1>II. The optimization problem</h1>
</div>

Logistic regression consists in finding the parameter $x \in \mathbb{R}^d$ that best fits:
$$
b_i = \begin{cases}
\phantom{-}1 \ \textrm{ if }\ \langle a_{i}, x\rangle \ge 0 \\
-1 \ \textrm{ if }\ \langle a_{i}, x\rangle < 0 
\end{cases}
$$
for all the samples $i$ of the training set (where $a_i$ is the $i^{th}$ line of $A$ and $b_i$ the $i^{th}$ element of $b$). To measure this fit, logistic regression chooses the following loss:

$$
\textrm{LogisticLoss}(b_i, \left\langle a_{i}, x\right\rangle ) = \log \left(1+\exp \left(-b_{i}\left\langle a_{i}, x\right\rangle\right)\right) 
$$

The corresponding optimization problem we will consider is the following:

<div style="background-color: #eceff1; border-left: 0px solid #78909c; padding: 2em; border-radius: 5px;"> 

Minimization of the empirical risk $f$ of the $L_2$-regularized logistic regression model, on the dataset $(A_{\textrm{train}},b_{\textrm{train}})$:

$$
\begin{aligned}
\textrm{min}_{x\in \mathbb{R}^d} \  f(x) 
\ &= \ \textrm{min}_{x\in \mathbb{R}^d} \ \frac{1}{n_{\textrm{train}}} \sum_{i=1}^{n_{\textrm{train}}} f_i(x) \\
&= \ \textrm{min}_{x\in \mathbb{R}^d}\frac{1}{n_{\textrm{train}}} \sum_{i=1}^{n_{\textrm{train}}} \textrm{LogisticLoss}(b_i, \left\langle a_{i}, x\right\rangle ) + \frac{l_2}{2}\|x\|^{2}_2,  
\end{aligned}
$$

where $l_2 \ge 0$ is the regularization parameter.
</div>

To simplify the notation, we'll stop indexing $A$, $b$ and $n$ with "$\textrm{train}$" in the following.

Note that the functions $f_i$s and $f$ are $C^1$ with $L$-Lischitz gradient, with $L \le \frac{\max_i\|a_i\|^2_2}{4} + l_2$ and we can therefore use the following value in our implementation:
```python
L = 0.25 * max(np.linalg.norm(A,2,axis=1))**2 + self.l2
```

<br>
<div style="color: #ffffff; background-color: #2979ff; border-bottom: 5px solid #004ecb; padding: 2em;"> 
<h1>III. Implementing the solvers</h1>
</div>

To implement gradient methods, we need an oracle that will compute both batch gradients $\nabla f$ and partial gradients $\nabla f_i$.

<div style="background-color: #e1f5fe;  border-left: 5px solid #2962ff; padding: 0.5em;"> 
<b style="color: #2962ff;">Question:</b>  In the code below, $\nabla f$ has already been implemented.
    <ul>
        <li> Complete <b>TODO (1)</b> below.</li>
        <li> Add your code to <b>\lib\linear_model.py</b>.</li>
    </ul>
</div>

```python
def grad(x, i=None):
    '''
    Gradient of the objective function at x.
    If no i is given, grad should return the batch gradient.
    Otherwise, it should return the partial gradient associated the the datapoint (A[i], b[i]).
    '''

    if i is None: # return batch gradient
        output = -b/(1 + np.exp(b*np.dot(A, x)))
        output = sum(np.diag(output))/n @ A
        output += self.l2*x
    else: # return partial gradient associated the datapoint (A[i], b[i])
        ######## TODO (1) ########
        pass
    return output
```

# GD (Gradient descent) 

At iteration k of the GD algorithm, the following update is executed:

$$
x_{k+1} \leftarrow x_{k} - \gamma\nabla f(x_k)
$$

<div style="background-color: #e1f5fe;  border-left: 5px solid #2962ff; padding: 0.5em;"> 
<b style="color: #2962ff;">Question:</b>  
    <ul>
        <li> What stepsize $\gamma$ ensures the convergence of this algorithm ?</li>
        <li> Complete <b>TODO (2)</b> and <b>TODO (3)</b></li>
        <li> Add your code to <b>\lib\solvers.py</b>.</li>
    </ul>
</div>

> **Answer:**
>
>

```python
def GD(x0, grad, max_iter, n, L, mu):
    '''
    Parameters
    ----------
    x0: array, shape (nb_features,)
        Initialisation of the solver
    grad: function
        Gradient of the objective function
    max_iter: int
        Number of iterations (i.e. number of descent steps). Note that for GD, 
        one iteration is one epoch i.e, one pass through the data, while for SGD, SAG and SAGA, 
        one iteration uses only one data point.
    n: int
        Dataset size
    L: float
        Smoothness constant of the objective function
    mu: float
        Strong convexity constant of the objective function
        
    Returns
    -------
    x: array, shape (nb_features,)
        final iterate of the solver
    x_tab: array, shape (nb_features, max_iter)
        table of all the iterates
    '''
    stepsize  = ######## TODO (2) ########
    x         = x0
    x_tab     = np.copy(x)

    for k in range(max_iter):
        x     = ######## TODO (3): Implement a single step ########
        x_tab = np.vstack((x_tab, x))

    return x, x_tab
```

<div style="background-color: #e1f5fe;  border-left: 5px solid #2962ff; padding: 0.5em;"> 
<b style="color: #2962ff;">Question:</b> Run the code below to test your code.
</div>

In [ ]:
from lib.linear_model import LogisticRegression
from lib.visuals import learning_curve

clf_gd = LogisticRegression(solver='gd', l2=0.01, max_iter=100)
clf_gd.fit(A_train, b_train)

learning_curve(GD=clf_gd)

# SGD (Stochastic Gradient Descent)

At iteration k of the SGD algorithm, the following update is executed:

$$
\begin{cases}
i \textrm{ chosen randomly in } \{1...n\}\\
x_{k+1} \leftarrow x_{k} - \gamma_k \nabla f_i(x_k) \\
\end{cases}
$$

<div style="background-color: #e1f5fe;  border-left: 5px solid #2962ff; padding: 0.5em;"> 
<b style="color: #2962ff;">Question:</b>  
    <ul>
        <li> What stepsize $\gamma_k$ ensures the convergence of this algorithm and under what assumptions?</li>
        <li> Complete <b>TODO (4)</b></li>
        <li> Add your code to <b>\lib\solvers.py</b>.</li>
    </ul>
</div>

```python
def SGD(x0, grad, max_iter, n, L, mu):
    ######## TODO (4) ########

    for k in range(max_iter):
        ######## TODO (4) ########
        
        if k%n == 0: # each completed epoch
            x_tab = np.vstack((x_tab, x))

    return x, x_tab
```

<div style="background-color: #e1f5fe;  border-left: 5px solid #2962ff; padding: 0.5em;"> 
<b style="color: #2962ff;">Question:</b> Run the code below to test your code.
</div>

In [ ]:
clf_sgd = LogisticRegression(solver='sgd', l2=0.01, max_iter=100*len(b_train))
clf_sgd.fit(A_train, b_train)

learning_curve(SGD=clf_sgd)

# SAGA (Stochastic Average Gradient "Amélioré")

SAGA starts by computing a single batch gradient, then at iteration k, the following update is executed:

$$
\begin{cases}
i \textrm{ chosen randomly in } \{1...n\}\\
x_{k+1} \leftarrow x_{k} - \gamma_k\big(\nabla f_i(x_k) - \alpha_i + \bar{\alpha}\big) \\
\alpha_i \leftarrow \nabla f_i(x) 
\end{cases}
$$

where $(\alpha_j)_{j=1}^n$ are gradients evaluated at previous iterates and $\bar{\alpha}=\frac{1}{n}\sum^n_{j=1}\alpha_j$.


<div style="background-color: #e1f5fe;  border-left: 5px solid #2962ff; padding: 0.5em;"> 
<b style="color: #2962ff;">Question:</b>  
    <ul>
        <li> What stepsize $\gamma_k$ ensures the convergence of this algorithm and under what assumptions?</li>
        <li> Complete <b>TODO (5)</b></li>
        <li> Add your code to <b>\lib\solvers.py</b>.</li>
    </ul>
</div>

```python
def SAGA(x0, grad, max_iter, n, L, mu):
    ######## TODO (5) ########

    for k in range(max_iter):
        ######## TODO (5) ########
        
        if k%n == 0: # each completed epoch
            x_tab = np.vstack((x_tab, x))

    return x, x_tab
```

<div style="background-color: #e1f5fe;  border-left: 5px solid #2962ff; padding: 0.5em;"> 
<b style="color: #2962ff;">Question:</b> Run the code below to test your code.
</div>

In [ ]:
clf_saga = LogisticRegression(solver='saga', l2=0.01, max_iter=30*len(b_train))
clf_saga.fit(A_train, b_train)

learning_curve(SAGA=clf_saga)

# Comparison

<div style="background-color: #e1f5fe;  border-left: 5px solid #2962ff; padding: 0.5em;"> 
<b style="color: #2962ff;">Question:</b> Run the code below to compare all the methods you've implemented.
</div>

In [ ]:
epochs = 50
n = len(b_train)

clf_gd   = LogisticRegression(solver='gd', l2=0.01, max_iter=epochs)
clf_sgd  = LogisticRegression(solver='sgd', l2=0.01, max_iter=epochs*n)
clf_saga = LogisticRegression(solver='saga', l2=0.01, max_iter=epochs*n)

clf_gd.fit(A_train, b_train)
clf_sgd.fit(A_train, b_train)
clf_saga.fit(A_train, b_train)

In [ ]:
learning_curve(GD=clf_gd, SGD=clf_sgd, SAGA=clf_saga)

for clf, name in zip([clf_gd, clf_sgd, clf_saga], ['gd', 'sgd', 'saga']):
    print(name + ':' \
          + "\tAccuracy on training set: {0:0.1f}%".format(100*clf.score(A_train, b_train)))
    print("\tAccuracy on testing set:  {0:0.1f}%".format(100*clf.score(A_test, b_test)))

<br>
<div style="color: #ffffff; background-color: #2979ff; border-bottom: 5px solid #004ecb; padding: 2em;"> 
<h1>IV. Regularization</h1>
</div>

<div style="background-color: #e1f5fe;  border-left: 5px solid #2962ff; padding: 0.5em;"> 
<b style="color: #2962ff;">Question:</b> 
    <ul>
        <li> Rewrite the empirical risk minimization objective with both $L_1$ and $L_2$ regularization (with coefficients $l_1$ and $l_2$).</li>
        <li> Is this objective $\mu$-strongly convex, $L$-smooth, differentiable?</i>
        <li> Complete the <b>TODO (6)</b> to account for the new term in the objective.</i>
        <li> Add your code to <b>lib/linear_model.py</b>.</i>
    </ul>
</div>

> **Answer:**
>

```python
self._empirical_risk = lambda x: sum(self._logistic_loss(A, b, x))/n \
                                 + self.l2/2.0 * np.linalg.norm(x,2) \
                                 + ######## TODO (6) ########
```

The minimization algorithms we've implemented have a slow rate of convergence for this new objective function. We will therefore use *proximal* variations of these algorithms.

<div style="background-color: #e1f5fe;  border-left: 5px solid #2962ff; padding: 0.5em;"> 
<b style="color: #2962ff;">Question:</b> Using the $\textrm{prox}_{l_1 \|\cdot\|_1}$ operator, write down the $k^{th}$ step of proximal GD, proximal SGD, and proximal SAGA.
</div>

> **Answer:**
>

<div style="background-color: #e1f5fe;  border-left: 5px solid #2962ff; padding: 0.5em;"> 
<b style="color: #2962ff;">Question:</b> 
    <ul>
        <li> Give a closed-form expression of $\textrm{prox}_{l_1 \|\cdot\|_1}$</li>
        <li> Complete <b>TODO (7)</b> (note that you can access the $l_1$ coefficient using <b>self.l1</b>).</li>
        <li> Add your code to <b>\lib\linear_model.py</b>.</li>
    </ul>
</div>

> **Answer:**
>

```python
def prox(x, stepsize):
    ######## TODO (7) ########
    pass
```

<div style="background-color: #e1f5fe;  border-left: 5px solid #2962ff; padding: 0.5em;"> 
<b style="color: #2962ff;">Question:</b> 
    <ul>
        <li> Add <b>prox</b> to the attributes of the solvers (both in the call to <b>self.solver</b> in <b>/lib/linear_model.py</b> and in <b>/lib/solvers.py</b>) as follows: <br> solver(x0, grad, <b>prox</b>, self.max_iter, n, L, mu)</li>
        <li> Add a proximal step to all the algorithms in <b>/lib/solvers.py</b>. </li>
        <li> Paste one of these algorithms in the cell bellow.</li>
    </ul>
</div>

```python
def {GD or SGD or SAGA}(x0, grad, prox, max_iter, n, L, mu):
```

<div style="background-color: #e1f5fe;  border-left: 5px solid #2962ff; padding: 0.5em;"> 
<b style="color: #2962ff;">Question:</b> If your proximal gradient is correctly implemented, the following cell plots the support of the solution for various values of $l_1$.
    <ul>
        <li> Interpret the plot and explain what can $L_1$ regularization be used for.</li>
    </ul>
</div>

> **Answer:**
>

In [ ]:
from lib.visuals import l1_regularization_plot

clf = lambda l1: LogisticRegression(solver='gd', l1=l1, l2=0.1, max_iter=1000)

l1_regularization_plot(clf, A_train, b_train)

<div style="background-color: #e1f5fe;  border-left: 5px solid #2962ff; padding: 0.5em;"> 
<b style="color: #2962ff;">Question:</b> The following cell will plot the value of the coordinates of the solution for various values of $l_2$.
    <ul>
        <li> Interpret the plot and explain what can $L_2$ regularization be used for.</li>
    </ul>
</div>

> **Answer:**
>

In [ ]:
from lib.visuals import l2_regularization_plot

clf = lambda l2: LogisticRegression(solver='gd', l1=0.01, l2=l2, max_iter=1000)

l2_regularization_plot(clf, A_train, b_train)

<br>
<div style="color: #ffffff; background-color: #2979ff; border-bottom: 5px solid #004ecb; padding: 2em;"> 
<h1>V. Further development</h1>
</div>

We expect that you go beyond these questions and that you develop certain aspects, depending on your personal interests and skills, for instance:
- Implementations and numerical tests (further developments, more experiments, etc.)
- Applications in learning or statistics (interpretation of some results, other models, other datasets, etc.)
- Theoretical or mathematical questions (convergence proofs, convergence rates, advanced versions of an algorithm, theoretical analysis of special cases, etc.) 